# Fit of the SIR patch model to CSSE Sand Data

[Index](0-index.ipynb)

## Imports and global variables

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from pathlib import Path
import os,sys
import numpy as np
import pandas as pd
import datetime
import scipy
from scipy.optimize import curve_fit

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import matplotlib.colors as mco
import matplotlib.gridspec as mgs
import matplotlib.cm as cm
from matplotlib.gridspec import GridSpec
from matplotlib import animation
plt.rcParams['svg.fonttype'] = 'none'

from IPython.display import HTML
from IPython.display import Image

In [ ]:
sys.path.append(str(Path('..') / 'code'))
from functions import fsigmoid, fsigmoid_jac, fit_sir

In [ ]:
resdir = Path('../results/')
if not resdir.is_dir():
    raise ValueError('No results directory!')

In [ ]:
resfile = resdir / 'safegraph_analysis.hdf5'
complevel=7
complib='zlib'
with pd.HDFStore(resfile, complevel=complevel, complib=complib) as store:
    print(f"File {resfile.stem} has {len(store.keys())} entries.")

## Global variables and other quantities

### Global variables

In [ ]:
gamma = 1/10.
ti = '2020-03-01'
tf = '2021-02-16'

tfmt = '%Y-%m-%d'
ti = datetime.datetime.strptime(ti, tfmt)
tf = datetime.datetime.strptime(tf, tfmt)

exts = ['.png', '.svg']

### Load clusters to get population

In [ ]:
key = "/clustering/clusters"
with pd.HDFStore(resfile, complevel=complevel, complib=complib) as store:
    clusters = store[key]
# clusters = pd.read_hdf(resfile, key)
N = len(clusters)
print(f"N = {N}")
clusters

In [ ]:
population = clusters['population'].to_numpy()
population_inv = np.zeros(population.shape, dtype=np.float_)
idx = population > 0.
population_inv[idx] = 1./population[idx]

### Load CSSEGI data

In [ ]:
path = '/clustering/cssegi'
with pd.HDFStore(resfile, complevel=complevel, complib=complib) as store:
    df_cssegi = store[path]

times = df_cssegi.index
idx = (times >= ti) & (times <= tf)
df_cssegi.drop(index=times[~idx], inplace=True)
times = df_cssegi.index.to_pydatetime().tolist()
df_cssegi

In [ ]:
omega_real = df_cssegi.to_numpy().astype('float64')
domega_real = np.diff(omega_real, axis=0)
domega_real = np.concatenate([omega_real[0].reshape(1,-1), domega_real], axis=0)

In [ ]:
# compute the real epidemic sizes per community through time
T_real = np.einsum('ta,a->ta', omega_real, population_inv)
dT_real = np.einsum('ta,a->ta', domega_real, population_inv)
T_tot_real = np.einsum('ta,a->t', T_real, population) / np.sum(population)
dT_tot_real = np.einsum('ta,a->t', dT_real, population) / np.sum(population)

Show the total epidemic size from real data

In [ ]:
plt.plot(times, T_tot_real)
plt.gca().set_yscale('log')

Show the initial condition

In [ ]:
X = clusters.index.to_numpy()
Y = np.einsum('ta,a->ta', T_real, population)[0]
idx = Y > 0
plt.plot(X[idx],Y[idx], 'bo')

## Prepare the infectivity matrices for the fit

In [ ]:
# global variable
pathtofit = Path('/fit')

In [ ]:
from functions import get_infectivity_matrix

In [ ]:
# first check that all times have an associated flux matrix
pathtoflux = '/fluxes'
time_fluxes = []
# read the mean flux matrix
with pd.HDFStore(resfile, complevel=complevel, complib=complib) as store:
    for rt, dirs, files in store.walk(pathtoflux):
        for f in files:
            t = datetime.datetime.strptime(f, tfmt)
            time_fluxes.append(t)

time_fluxes.sort()

for t in times:
    if not t in time_fluxes:
        raise ValueError("Some dates don't have an associated flux matrix")

In [ ]:
# Method 1 - Averaging the flux matrix
pathtoflux = Path('/fluxes')

N = len(clusters)
L = np.zeros((N,N), dtype=np.float_)
with pd.HDFStore(resfile, complevel=complevel, complib=complib) as store:
    for t in times:
        mykey = pathtoflux / t.strftime(tfmt)
        mykey = str(mykey)
        df_flux = store[mykey]
        L += df_flux.to_numpy().astype('float64')
L /= len(times)
L = get_infectivity_matrix(L)
df_loc = pd.DataFrame(data=L, index=clusters.index, columns=clusters.index)

pathtoloc = pathtofit / 'infectivity_matrices'
with pd.HDFStore(resfile, complevel=complevel, complib=complib) as store:
    mykey = str(pathtoloc / times[0].strftime(tfmt))
    store[mykey] = df_loc

In [ ]:
# export infectivity matrix (unscaled)
expdir = resdir / 'csv'
if not expdir.is_dir():
    expdir.mkdir()

fname = 'infectivity_mean.csv'
df_loc.to_csv(expdir / fname)

In [ ]:
# # Method 2 - Averaging the infectivity matrices
# pathtoflux = Path('/fluxes')

# N = len(clusters)
# L = np.zeros((N,N), dtype=np.float_)
# with pd.HDFStore(resfile, complevel=complevel, complib=complib) as store:
#     for t in times:
#         mykey = pathtoflux / t.strftime(tfmt)
#         mykey = str(mykey)
#         df_flux = store[mykey]
#         L += get_infectivity_matrix(df_flux.to_numpy().astype('float64'))
# L /= len(times)
# df_loc = pd.DataFrame(data=L, index=clusters.index, columns=clusters.index)

# pathtoloc = pathtofit / 'infectivity_matrices'
# with pd.HDFStore(resfile, complevel=complevel, complib=complib) as store:
#     mykey = str(pathtoloc / times[0].strftime(tfmt))
#     store[mykey] = df_loc

## Perform the fit

In [ ]:
print("Reading infectivity matrices from: {:s}".format(str(pathtoloc)))
with pd.HDFStore(resfile, complevel=complevel, complib=complib) as store:
    df_S, df_I, df_fit = fit_sir(times, T_real, gamma, population, store, pathtoloc, tfmt=tfmt, verbose=True)

path = pathtofit / 'result'
with pd.HDFStore(resfile, complevel=complevel, complib=complib) as store:
    mykey = str(path / 'susceptible')
    store[mykey] = df_S
    
    mykey = str(path / 'infected')
    store[mykey] = df_I
    
    mykey = str(path / 'fit')
    store[mykey] = df_fit

## Show agreement with CSSEGI data

In [ ]:
figdir = Path('../figures') / '5-SIR_dynamics_fit'
if not figdir.is_dir():
    figdir.mkdir(parents=True, exist_ok=True)

In [ ]:
path = pathtofit / 'result'
with pd.HDFStore(resfile, complevel=complevel, complib=complib) as store:
    mykey = str(path / 'susceptible')
    df_S = store[mykey]
    
    mykey = str(path / 'infected')
    df_I = store[mykey]

In [ ]:
idx = df_S.index.to_pydatetime().tolist()
idx.sort()
if (idx != times):
    raise ValueError("Incompatible times!")

S = df_S.to_numpy().astype('float64')
T = (1. - S)
dT = np.diff(T, axis=0)
dT = np.concatenate([T[0].reshape(1,-1), dT], axis=0)

df_T = pd.DataFrame(data=T, index=times, columns=df_S.columns)
df_dT = pd.DataFrame(data=dT, index=times, columns=df_S.columns)

T_tot = np.einsum('ta,a', T, population) / np.sum(population)
dT_tot = np.einsum('ta,a', dT, population) / np.sum(population)

In [ ]:
# parameters
figsize = (6,4.5)
dpi = 300
ms=2
lw=1
show_dT=True


fig = plt.figure(facecolor='w', figsize=figsize)
ax = fig.gca()

if show_dT:
    ax.plot(times,dT_tot, '-', ms=ms, color='darkblue')
    ax.plot(times,dT_tot_real, 'o', lw=lw, color='red')
    ax.set_ylabel("$d T$", fontsize="medium")
    fname = 'domega_tot_fit'
else:
    ax.plot(times,omega_tot, 'o', ms=ms, color='darkblue')
    ax.plot(times,omega_tot_real, '-', lw=lw, color='red')
    ax.set_ylabel("$T$", fontsize="medium")
    ax.set_yscale('log')
    fname = 'omega_tot_fit'

ax.set_xlim(times[0],None)
plt.xticks(rotation=45)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.tick_params(left=True, labelleft=True, bottom=True, labelbottom=True)
ax.tick_params(axis='both', length=4)
fig.tight_layout()

for ext in exts:
    filepath = figdir / (fname + ext)
    fig.savefig(filepath, bbox_inches='tight', pad_inches=0, dpi=dpi)
    print("Written file: {:s}".format(str(filepath)))
fig.clf()
plt.close('all')

In [ ]:
filepath = figdir / (fname + '.png')
Image(filename=filepath, width=4./3*360)

## Fit the scale

In [ ]:
path = pathtofit / 'result'
with pd.HDFStore(resfile, complevel=complevel, complib=complib) as store:
    mykey = str(path / 'fit')
    df_fit = store[mykey]

In [ ]:
# fit to sigmoid
df_idx = df_fit.dropna().index
X = (df_idx - df_idx[0]).days.to_numpy().astype('float64')
Y = df_fit.loc[df_idx, "scale"].to_numpy().astype('float64')

popt, pcov = curve_fit(fsigmoid, X, Y, p0=[0.,0.,0.,0.], jac=fsigmoid_jac, method='lm')

Ysigmoid = fsigmoid(X, *popt).astype('float64')

# build step function
a, b, c, d = popt
A = a+c
B = c
x0 = int(np.round(b)) # must be an integer to represent a day

Ystep = np.ones(len(X), dtype=np.float_)*A
idx = X>=x0
Ystep[idx] = B

In [ ]:
df_fit['scale_sigmoid'] = None
df_fit.loc[df_idx, 'scale_sigmoid'] = Ysigmoid

df_fit['scale_step'] = None
df_fit.loc[df_idx, 'scale_step'] = Ystep

path = pathtofit / 'result'
with pd.HDFStore(resfile, complevel=complevel, complib=complib) as store:    
    mykey = str(path / 'fit')
    store[mykey] = df_fit

expdir = resdir / 'csv'
if not expdir.is_dir():
    expdir.mkdir()

fname = 'fit_scale.csv'
df_fit.to_csv(expdir / fname)

In [ ]:
path = pathtofit / 'result'
with pd.HDFStore(resfile, complevel=complevel, complib=complib) as store:
    mykey = str(path / 'fit')
    df_fit = store[mykey]
    
df_fit

In [ ]:
# parameters
figsize = (8,4.5)
dpi = 300
ms=2
lw=1

X = df_fit.dropna().index
Y, Ysigmoid, Ystep = df_fit.dropna().loc[:,['scale', 'scale_sigmoid','scale_step']].to_numpy().astype('float64').T

fig = plt.figure(facecolor='w', figsize=figsize)
ax = fig.gca()

ax.plot(X,Y, 'o', ms=ms, color='k')
ax.plot(X,Ysigmoid, '-', lw=lw, color='darkblue')
ax.plot(X,Ystep, '-', lw=lw, color='darkgreen')

ax.set_xlim(X[0],None)
# ax.set_ylim(1.0e-7, 1.0e-1)
ax.set_xlabel("date", fontsize="medium")
ax.set_ylabel("$p \\beta$", fontsize="medium")
plt.xticks(rotation=45)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.tick_params(left=True, labelleft=True, bottom=True, labelbottom=True)
ax.tick_params(axis='both', length=4)
fig.tight_layout()

fname = 'scales'
for ext in exts:
    filepath = figdir / (fname + ext)
    fig.savefig(filepath, bbox_inches='tight', pad_inches=0, dpi=dpi)
    print("Written file: {:s}".format(str(filepath)))
fig.clf()
plt.close('all')

In [ ]:
filepath = figdir / (fname + '.png')
Image(filename=filepath, width=4./3*360)

Retrieving the shift-time

In [ ]:
b_scales = df_fit['scale_step'].fillna(value=np.nan).to_numpy()
ic = np.nanargmax(np.abs(np.diff(b_scales))) + 1
tc = df_fit.index[ic]
print(f"lockdown at t = {tc}")

In [ ]:
df_fit.iloc[ic-1:ic+2]

## Show local agreement between reported cases and the model

In [ ]:
from dateutil.relativedelta import relativedelta

Show $T_a$

In [ ]:
# parameters
vmin = 1.0e0
vmax = 1.0e6
cmap = cm.rainbow
figsize=(4,3)
dpi=300

## color mapping with date value
indices = np.arange(len(times))
norm = mco.Normalize(vmin=np.min(indices), vmax=np.max(indices))
colors = cmap(norm(indices))

## make figure
fig = plt.figure(facecolor='w', figsize=figsize, dpi=dpi)
ax = fig.gca()

for i in range(len(times)):
    t = times[i]
    X = T_real[i]*population
    Y = T[i]*population
    
    ax.plot(X, Y, 'o', color=colors[i], lw=0, mew=0, ms=2, alpha=0.1)    

ax.plot([vmin, vmax], [vmin, vmax], 'k-', lw=0.5)
# plot formatting
ax.set_xlabel("$T_a^{real}$", fontsize='medium')
ax.set_ylabel("$T_a$", fontsize='medium')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.tick_params(bottom=True, left=True, labelbottom=True, labelleft=True)
ax.tick_params(length=4)
ax.set_xlim(vmin, vmax)
ax.set_ylim(vmin, vmax)
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_aspect('equal')
        
    
fig.tight_layout(rect=[0.,0.,0.95,1.])
cax = fig.add_axes(rect=[0.99,0.2,0.01,0.7])
cbar = plt.colorbar(cm.ScalarMappable(norm=norm, cmap=cmap),cax=cax, extendfrac='auto')
nmonth = (times[-1].year-times[0].year)*12 + (times[-1].month-times[0].month)
tick_values = np.array([times[0] + relativedelta(months=i) for i in range(nmonth+1)])
tick_values = tick_values[::2]
ticks = [times.index(t) for t in tick_values]
# labels = [times[(t-1)*window].strftime('%Y-%m-%d') for t in np.array(ticks, dtype=np.int_)]
labels = [t.strftime('%Y-%m-%d') for t in tick_values]
cbar.set_ticks(ticks)
cbar.set_ticklabels(labels)

fname = 'T_model_vs_real'
for ext in exts:
    filepath = figdir / (fname + ext)
    fig.savefig(filepath, bbox_inches='tight', pad_inches=0, dpi=dpi)
    print("Written file: {:s}".format(str(filepath)))
fig.clf()
plt.close('all')
# plt.show()

In [ ]:
filepath = figdir / (fname + '.png')
Image(filename=filepath, width=4./3*360)

Show $dT_a$

In [ ]:
# parameters
vmin = 1.0e0
vmax = 1.0e4
cmap = cm.rainbow
figsize=(4,3)
dpi=300

## color mapping with date value
indices = np.arange(len(times))
norm = mco.Normalize(vmin=np.min(indices), vmax=np.max(indices))
colors = cmap(norm(indices))

## make figure
fig = plt.figure(facecolor='w', figsize=figsize, dpi=dpi)
ax = fig.gca()

for i in range(len(times)):
    t = times[i]
    X = dT_real[i]*population
    Y = dT[i]*population
    
    ax.plot(X, Y, 'o', color=colors[i], lw=0, mew=0, ms=2, alpha=0.1)    

ax.plot([vmin, vmax], [vmin, vmax], 'k-', lw=0.5)
# plot formatting
ax.set_xlabel("$dT_a^{real}$", fontsize='medium')
ax.set_ylabel("$dT_a$", fontsize='medium')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.tick_params(bottom=True, left=True, labelbottom=True, labelleft=True)
ax.tick_params(length=4)
ax.set_xlim(vmin, vmax)
ax.set_ylim(vmin, vmax)
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_aspect('equal')
    
fig.tight_layout(rect=[0.,0.,0.95,1.])
cax = fig.add_axes(rect=[0.99,0.2,0.01,0.7])
cbar = plt.colorbar(cm.ScalarMappable(norm=norm, cmap=cmap),cax=cax, extendfrac='auto')
nmonth = (times[-1].year-times[0].year)*12 + (times[-1].month-times[0].month)
tick_values = np.array([times[0] + relativedelta(months=i) for i in range(nmonth+1)])
tick_values = tick_values[::2]
ticks = [times.index(t) for t in tick_values]

labels = [t.strftime('%Y-%m-%d') for t in tick_values]
cbar.set_ticks(ticks)
cbar.set_ticklabels(labels)


fname = 'dT_model_vs_real'
for ext in exts:
    filepath = figdir / (fname + ext)
    fig.savefig(filepath, bbox_inches='tight', pad_inches=0, dpi=dpi)
    print("Written file: {:s}".format(str(filepath)))
fig.clf()
plt.close('all')
# plt.show()

In [ ]:
filepath = figdir / (fname + '.png')
Image(filename=filepath, width=4./3*360)

## Show time-dependent profile

In [ ]:
from functions import plot_omega_profile

In [ ]:
# parameters
dpi=150
fps=10
figsize=(6, 4.5)
lw=0.5
ms=4

mydir = figdir / 'profiles'
if not mydir.is_dir():
    mydir.mkdir(parents=True, exist_ok=True)

fpath = mydir / 'profile_T.mp4'
T_list = np.array([np.einsum('ta,a->ta', T_real[:,:],population), np.einsum('ta,a->ta', T[:,:], population)])
ylabel="$T_a$"
# T_list = np.array([np.einsum('ta,a->ta', domega_real, population), np.einsum('ta,a->ta', dT, population)]) / np.sum(population)
plot_omega_profile(T_list, times, labels=['real', 'model'], colors=['red', 'darkblue'], \
                   fileout=fpath, tpdir=mydir / 'snapshots_T', dpi=dpi, fps=fps, figsize=figsize, ylabel=ylabel, \
                   lw=lw, ms=ms, styles=['o', '-'], deletetp=False, exts=['.png','.svg'], ymin=1.)

fpath = mydir / 'profile_dT.mp4'
T_list = np.array([np.einsum('ta,a->ta', dT_real[:,:],population), np.einsum('ta,a->ta', dT[:,:], population)])
ylabel="$dT_a$"
plot_omega_profile(T_list, times, labels=['real', 'model'], colors=['red', 'darkblue'], \
                   fileout=fpath, tpdir=mydir / 'snapshots_dT', dpi=dpi, fps=fps, figsize=figsize, ylabel=ylabel, \
                   lw=lw, ms=ms, styles=['o', '-'], deletetp=False, exts=['.png','.svg'], ymin=1.)

In [ ]:
fpath = figdir / 'profiles' / 'profile_T.mp4'
HTML("""
<video height="360" controls>
  <source src="{:s}" type="video/mp4">
</video>
""".format(str(fpath)))

In [ ]:
fpath = figdir / 'profiles' / 'profile_dT.mp4'
HTML("""
<video height="360" controls>
  <source src="{:s}" type="video/mp4">
</video>
""".format(str(fpath)))

## Show time-dependent map

In [ ]:
from functions import plot_omega_map

In [ ]:
# parameters
dpi=150
fps=10
figsize=(6, 4.5)
lw=0.5
ms=4
idump=1

mydir = figdir / 'maps'
if not mydir.is_dir():
    mydir.mkdir(parents=True, exist_ok=True)


fpath = mydir / 'map_T.mp4'
plot_omega_map(np.einsum('ta,a->ta', T, population), times, XY=clusters.loc[:, ['X', 'Y']].to_numpy().T, \
fileout=fpath, tpdir=mydir / 'snapshots_T', dpi=dpi, fps=fps, figsize=figsize, idump=idump, \
               clabel="$T$", vmin=1., deletetp=False, exts=['.png','.svg'])
    
fpath = mydir / 'map_dT.mp4'
plot_omega_map(np.einsum('ta,a->ta', dT, population), times, XY=clusters.loc[:, ['X', 'Y']].to_numpy().T, \
fileout=fpath, tpdir=mydir / 'snapshots_dT', dpi=dpi, fps=fps, figsize=figsize, idump=idump, \
               clabel="$dT$", vmin=1., deletetp=False, exts=['.png','.svg'])

In [ ]:
fpath = figdir / 'maps' / 'map_T.mp4'
HTML("""
<video height="360" controls>
  <source src="{:s}" type="video/mp4">
</video>
""".format(str(fpath)))

In [ ]:
fpath = figdir / 'maps' / 'map_dT.mp4'
HTML("""
<video height="360" controls>
  <source src="{:s}" type="video/mp4">
</video>
""".format(str(fpath)))